download pretrained word2vec embeddings from https://code.google.com/archive/p/word2vec/

In [0]:
import sys
import os
!pip install gensim pyspellchecker
# sys.path.append(os.path.abspath('/Workspace/Users/rischand@publicisgroupe.net/Temp/chatbot/'))

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 95.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import os
import re
import numpy as np
import gensim
import pandas as pd
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
import tornado
import tornado.httpserver
import tornado.ioloop
from tornado import web
import pickle
import yaml
import requests
import traceback
import json
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker

# Global vars:
global QUESTION_BANK, WV_EMBEDDINGS
WV_EMBEDDINGS  = None    # = google embeddings : is a 5GB data structure, hence hold only 1 instance
QUESTION_BANK  = None    # = Question_Bank() : holds all the question bank data, hence hold only 1 instance

# for text cleaning
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

# for word replacements
REPLACE = {
    'corona'              : 'virus',
    'coronavirus'         : 'virus',
    'covid'               : 'virus',
    'covid-19'            : 'virus',    
}
DICT_FILE = './EnglishWords.txt'
SPELL = SpellChecker()
SPELL.word_frequency.load_text_file (DICT_FILE)

# optional vocabulary
EMB_VOCAB_SET = None
with open (DICT_FILE, 'r') as f:
    EMB_VOCAB_SET = set (f.readlines())
EMB_VOCAB_SET = {w.strip() for w in EMB_VOCAB_SET}


def fix_word (word, embeddings=None, isStemming=False, isCorrection=True):
    
    word = str (word)
    if word in embeddings:
        return word
    
    stemmer = PorterStemmer ()
    lemmatizer = WordNetLemmatizer ()
    word = word.lower ().strip ()
    if embeddings:
        if not word in embeddings:
            word = lemmatizer.lemmatize (word)
            if isStemming:
                word = stemmer.stem (word)
            if isCorrection:
                temp_word = SPELL.correction (word)
                if temp_word:
                    word = temp_word
                else:
                    word = ''
    else:
        word = lemmatizer.lemmatize (word)
        if isStemming:
            word = stemmer.stem (word)
        if isCorrection and not word in EMB_VOCAB_SET:
            temp_word = SPELL.correction (word)
            if temp_word:
                word = temp_word
            else:
                word = ''
    return word


def text_prepare (text, embeddings=None, isStemming=True, isCorrection=True):
    """
    text: a raw string        
    return: list of strings: modified list of words of the string
    """
    
    text = str (text)
    text = re.sub (r'http[^\s]*', '', text)
    text = re.sub (r'<mm/dd/yyyy>', '', text)
    text = re.sub (r'@[^\s]*', '', text)
    # text = re.sub (r'#\s?[^\s]*', '', text)
    text = text.lower () # lowercase text
    text = re.sub (REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub (BAD_SYMBOLS_RE, "", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    word_tokens = word_tokenize (text)
    # delete stopwords from text
    words = [fix_word (w, embeddings, isStemming, isCorrection) for w in word_tokens if not w in STOPWORDS]
    words = [word for word in words if word not in {'', ' '}]
    return words


def sent2vec_wv (sent, embeddings, dim, isStemming, isCorrection):
    """
    sent       : one raw sentence (string)
    embeddings : embeddings dict e.g. WV_EMBEDDINGS 
    dim        : (int) dimention of a word vector (= 300 as in the google's pretrained file)
    """
    
    sent = str (sent)
    result = np.zeros (dim)
    if not sent is None and len (sent)>0:
        
        cnt = 0     
        words = text_prepare (sent, embeddings, isStemming, isCorrection)
        for word in words:
            
            if word in embeddings:
                result += np.array (embeddings[word])
                cnt += 1
        if cnt != 0:
            result /= cnt
    return result


# Use Pretrained Google Word2Vec Embeddings
class WVEmbeddingTransformer (BaseEstimator, TransformerMixin):
    
    def __init__ (self, embFile=None, isStemming=False, isCorrection=True):
        """
        embFile  : str, filePath
        vocabSet : Set of words (strings). if not None then self.isinit=True else False. 
                    If self.isinit==True then .fit() will do nothing, even when params are provided.
        init:
            self.dim    : int
            self.isinit : boolean
        """

        self.isStemming = isStemming
        self.isCorrection = isCorrection
        self.embFile = embFile
        self.isTransformInit = False
        self.spellChecker = None
        self.vocabSet = None
        self.embedding = WV_EMBEDDINGS
        self.dim = 0

    
    def fit (self, Xstr, Y=None, **fit_params):
        """
        Xstr : set[str] or list[str], set of raw words
        """
        
        global WV_EMBEDDINGS, SPELL, EMB_VOCAB_SET
        vocabSet = set(Xstr)
        WV_EMBEDDINGS = KeyedVectors.load_word2vec_format (datapath (self.embFile), binary=True)

        # inint the vocab to all embeddings.vocab
        with open(DICT_FILE, "w") as f:
            for w in WV_EMBEDDINGS.key_to_index:
                f.write(f"{w}\n")
        SPELL = SpellChecker()
        SPELL.word_frequency.load_text_file (DICT_FILE)
        if vocabSet and len (vocabSet)>0:
            
            # set the vocab to the given reduced custom vocab
            vocabSet = text_prepare(' '.join(vocabSet), WV_EMBEDDINGS, self.isStemming, self.isCorrection)
            temp = {w:WV_EMBEDDINGS[w] for w in vocabSet if w in WV_EMBEDDINGS}
            WV_EMBEDDINGS = temp

            with open(DICT_FILE, "w") as f:
                for w in vocabSet:
                    f.write(f"{w}\n")
            SPELL = SpellChecker()
            SPELL.word_frequency.load_text_file (DICT_FILE)
            EMB_VOCAB_SET = vocabSet
        
        # fix dimension
        dim = 0
        if not type(WV_EMBEDDINGS) is dict:
            for w in WV_EMBEDDINGS.key_to_index:
                dim = len(WV_EMBEDDINGS[w])
                break
        else:
            for w in WV_EMBEDDINGS:
                dim = len(WV_EMBEDDINGS[w])
                break
            
        self.dim = dim
        self.vocabSet = vocabSet
        self.spellChecker = SPELL
        self.embedding = WV_EMBEDDINGS

        # print("vocabSet =", vocabSet)
        # print("SPELL =", SPELL)
        # print("WV_EMBEDDINGS words =", WV_EMBEDDINGS.keys())
        print(f"embeddings shape = [{len(WV_EMBEDDINGS)}, {dim}]")
    
    
    def transform (self, Xstr, Y=None, **fit_params):
        """
        Xstr   : list of sentences (strings)
        return : np.array(list of corresponding sentence embeddings (avg. of word embeddings of the sent))
        """

        global WV_EMBEDDINGS, SPELL, EMB_VOCAB_SET
        if not self.isTransformInit:
            SPELL = self.spellChecker
            EMB_VOCAB_SET = self.vocabSet
            WV_EMBEDDINGS = self.embedding
            self.isTransformInit = True
            
        # print("self.vocabSet =", self.vocabSet)
        # print("self.spellChecker =", self.spellChecker)
        # print("self.embedding words =", self.embedding.keys())
        # print(f"embeddings shape = [{len(self.embedding)}, {self.dim}]")

        X = [sent2vec_wv (sent, self.embedding, self.dim, self.isStemming, self.isCorrection) for sent in Xstr]
        return np.array(X)


# Nearest Neighbour Question Bank
def getCosineNearestNeighbours (q_vec, cand_vecs, k=1, dim=300):
    """
    q_vec : 1 array vec of shape 1*dim
    cand_vecs : an array of N vecs each vec is a row
    returns the index positions of the k closest condidate vecs ones having highest cosine similarity
    """
    
    q_vec       = q_vec.reshape ((1, -1))
    cand_vecs   = cand_vecs.reshape ((-1, dim))
    cosines     = np.array (cosine_similarity (q_vec, cand_vecs)[0])
    merged_list = list (zip (cosines, range (len (cand_vecs))))
    # print(merged_list)
    sorted_list = sorted (merged_list, key=lambda x: x[0], reverse=True)
    result = [b for a,b in sorted_list]
    if len(result) > k:
        result = result[:k]
    return result


class Question_Bank:
    
    def __init__ (self, questionBank_filename, emb_file, vocabSet, isCorrection=True):
        """
        file should contain columns 'questions', 'answers', 'contexts': are raw text
        text cleaning is done on the fly while creating embeddings.
        """

        # vectorize the data and store it in mem nearest neighbor 
        self.df = pd.read_parquet (questionBank_filename)
        self.N  = self.df.shape[0]
        Xstr = list (self.df['questions']) + list (self.df['answers']) + list (self.df['contexts'])
        self.embTransformer = WVEmbeddingTransformer (emb_file, isCorrection=isCorrection)
        self.embTransformer.fit(vocabSet)
        self.CAND_VECS = self.embTransformer.transform (Xstr)
        return
    
    def get_nearest_QA_indices (self, qn_str):
        
        qn_str = str (qn_str)
        q_vec = self.embTransformer.transform ([qn_str])
        nearest_indices = getCosineNearestNeighbours (q_vec, self.CAND_VECS, k=4)
        qn_idxs = []
        for i in nearest_indices:
            
            if i+1 > 2*self.N:
                if i-2*self.N not in qn_idxs:
                    qn_idxs.append (i-2*self.N)
            elif i+1 > self.N:
                if i-self.N not in qn_idxs:
                    qn_idxs.append (i-self.N)
            elif i>=0 and i not in qn_idxs:
                qn_idxs.append (i)
        return qn_idxs

    def ask (self, qn_str):
        
        qn_str = str (qn_str)
        qn_idxs = self.get_nearest_QA_indices (qn_str)
        df = self.df.iloc[qn_idxs].reset_index (drop=True)
        question = 'NOT_FOUND'
        answer = 'NOT_FOUND'
        suggestions = []
        if df.shape[0]>0:
            question = str (df["questions"][0])
            answer = str (df["answers"][0])
        if df.shape[0]>1:
            suggestions = df.loc[1:, ["questions", "answers"]].reset_index(drop=True)
        return question, answer, suggestions


class Text_cleaner_transformer (BaseEstimator, TransformerMixin):
    
    def __init__(self, isStemming=True, isCorrection=False, embedding=None):
        
        self.isStemming   = isStemming
        self.isCorrection = isCorrection
        self.embedding    = embedding
        return
    
    def fit (self, Xstr, y=None, **fit_params):
        return self
    
    def transform (self, Xstr, y=None, **fit_params):
        
        Xstr   = [' '.join (text_prepare (sent, self.embedding, self.isStemming, self.isCorrection)) for sent in Xstr]
        return Xstr


def preProcess (rawfile, newFileName, column_names):
    
    df = pd.read_csv (rawfile)
    for column_name in column_names:
        Xstr = df[column_name]
        text_cleaner = Text_cleaner_transformer (isCorrection=True, embedding=WV_EMBEDDINGS)
        Xstr = text_cleaner.transform (Xstr)
        df[f"{column_name}_cleaned"] = Xstr

    df.to_csv (newFileName)
    return


def pickleSave (obj, file):    
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):
    return pickle.load (open (file, 'rb'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [0]:
df = pd.read_parquet("/dbfs/FileStore/surajit/train_00000_of_00001_c38b6c63d6178c71.parquet").head(100)
list(df.columns)
df.columns = [colname.lower() for colname in list(df.columns)]
df.to_parquet("/dbfs/FileStore/surajit/qa.parquet")
df

,contexts,questions,answers
0,The small intestine is part of the\n\t\t body...,What is (are) Small Intestine Cancer ?,Key Points\n - Small intest...
1,The small intestine is part of the\n\t\t body...,Who is at risk for Small Intestine Cancer? ?,Diet and health history can affect the risk of...
2,The small intestine is part of the\n\t\t body...,What are the symptoms of Small Intestine Cancer ?,Signs and symptoms of small intestine cancer i...
3,The small intestine is part of the\n\t\t body...,How to diagnose Small Intestine Cancer ?,Tests that examine the small intestine are use...
4,The small intestine is part of the\n\t\t body...,What is the outlook for Small Intestine Cancer ?,Certain factors affect prognosis (chance of re...
5,The small intestine is part of the\n\t\t body...,What are the stages of Small Intestine Cancer ?,Key Points\n - Tests and pr...
6,The small intestine is part of the\n\t\t body...,What are the treatments for Small Intestine Ca...,Key Points\n - There are di...
7,Renal cell cancer (also called kidney cancer ...,What is (are) Renal Cell Cancer ?,Key Points\n - Renal cell c...
8,Renal cell cancer (also called kidney cancer ...,What are the stages of Renal Cell Cancer ?,Key Points\n - After renal ...
9,Renal cell cancer (also called kidney cancer ...,What are the treatments for Renal Cell Cancer ?,Key Points\n - There are di...


In [0]:
vocab_set = set()
for colname in df.columns:
    vocab_set.update({w.lower() for w in list(df[colname].str.split(' ', expand=True).stack().unique())})

In [0]:
qb = Question_Bank(
    '/dbfs/FileStore/surajit/qa.parquet',
    '/dbfs/FileStore/surajit/GoogleNews_vectors_negative300_bin.gz',
    vocabSet=vocab_set,
)

embeddings shape = [786, 300]


In [0]:
artifacts = {
    "WV_EMBEDDINGS": WV_EMBEDDINGS,
    "EMB_VOCAB_SET": EMB_VOCAB_SET
}
pickleSave(artifacts, '/dbfs/FileStore/surajit/artifacts.bin')
pickleSave(qb, '/dbfs/FileStore/surajit/qb.bin')

In [0]:
question, answer, suggestions = qb.ask('explain intestine cancer')
print("Matching question =", question)
print("Answer =", answer)
print("\nMore Suggestions:")
suggestions.display()

Matching question = What is (are) Small Intestine Cancer ?
Answer = Key Points
                    - Small intestine cancer is a rare disease in which malignant (cancer) cells form in the tissues of the small intestine.    - There are five types of small intestine cancer.    - Diet and health history can affect the risk of developing small intestine cancer.    - Signs and symptoms of small intestine cancer include unexplained weight loss and abdominal pain.    - Tests that examine the small intestine are used to detect (find), diagnose, and stage small intestine cancer.    - Certain factors affect prognosis (chance of recovery) and treatment options.
                
                
                    Small intestine cancer is a rare disease in which malignant (cancer) cells form in the tissues of the small intestine.
                    The small intestine is part of the bodys digestive system, which also includes the esophagus, stomach, and large intestine. The digestive system rem

questions answers What are the treatments for Small Intestine Cancer ? Key Points
 - There are different types of treatment for patients with small intestine cancer. - Three types of standard treatment are used: - Surgery - Radiation therapy - Chemotherapy - New types of treatment are being tested in clinical trials. - Biologic therapy - Radiation therapy with radiosensitizers - Patients may want to think about taking part in a clinical trial. - Patients can enter clinical trials before, during, or after starting their cancer treatment. - Follow-up tests may be needed.
 
 
 There are different types of treatment for patients with small intestine cancer.
 Different types of treatments are available for patients with small intestine cancer. Some treatments are standard (the currently used treatment), and some are being tested in clinical trials. A treatment clinical trial is a research study meant to help improve current treatments or obtain information on new treatments for patients with cancer. When clinical trials show that a new treatment is better than the standard treatment, the new treatment may become the standard treatment. Patients may want to think about taking part in a clinical trial. Some clinical trials are open only to patients who have not started treatment.
 
 
 Three types of standard treatment are used:
 Surgery Surgery is the most common treatment of small intestine cancer. One of the following types of surgery may be done: - Resection: Surgery to remove part or all of an organ that contains cancer. The resection may include the small intestine and nearby organs (if the cancer has spread). The doctor may remove the section of the small intestine that contains cancer and perform an anastomosis (joining the cut ends of the intestine together). The doctor will usually remove lymph nodes near the small intestine and examine them under a microscope to see whether they contain cancer. - Bypass: Surgery to allow food in the small intestine to go around (bypass) a tumor that is blocking the intestine but cannot be removed. Even if the doctor removes all the cancer that can be seen at the time of the surgery, some patients may be given radiation therapy after surgery to kill any cancer cells that are left. Treatment given after the surgery, to lower the risk that the cancer will come back, is called adjuvant therapy. Radiation therapy Radiation therapy is a cancer treatment that uses high-energy x-rays or other types of radiation to kill cancer cells or keep them from growing. There are two types of radiation therapy: - External radiation therapy uses a machine outside the body to send radiation toward the cancer. - Internal radiation therapy uses a radioactive substance sealed in needles, seeds, wires, or catheters that are placed directly into or near the cancer. The way the radiation therapy is given depends on the type of the cancer being treated. External radiation therapy is used to treat small intestine cancer. Chemotherapy Chemotherapy is a cancer treatment that uses drugs to stop the growth of cancer cells, either by killing the cells or by stopping them from dividing. When chemotherapy is taken by mouth or injected into a vein or muscle, the drugs enter the bloodstream and can reach cancer cells throughout the body (systemic chemotherapy). When chemotherapy is placed directly into the cerebrospinal fluid, an organ, or a body cavity such as the abdomen, the drugs mainly affect cancer cells in those areas (regional chemotherapy). The way the chemotherapy is given depends on the type and stage of the cancer being treated.
 
 
 New types of treatment are being tested in clinical trials.
 This summary section describes treatments that are being studied in clinical trials. It may not mention every new treatment being studied. Information about clinical trials is available from the NCI website. Biologic therapy Biologic therapy is a treatment that uses the patient's immune system to fight cancer. Substances made b

In [0]:
qb = unpickle ('/dbfs/FileStore/surajit/qb.bin')

question, answer, suggestions = qb.ask('explain intestine cancer')
print("Matching question =", question)
print("Answer =", answer)
print("\nMore Suggestions:")
suggestions.display()

Matching question = What is (are) Small Intestine Cancer ?
Answer = Key Points
                    - Small intestine cancer is a rare disease in which malignant (cancer) cells form in the tissues of the small intestine.    - There are five types of small intestine cancer.    - Diet and health history can affect the risk of developing small intestine cancer.    - Signs and symptoms of small intestine cancer include unexplained weight loss and abdominal pain.    - Tests that examine the small intestine are used to detect (find), diagnose, and stage small intestine cancer.    - Certain factors affect prognosis (chance of recovery) and treatment options.
                
                
                    Small intestine cancer is a rare disease in which malignant (cancer) cells form in the tissues of the small intestine.
                    The small intestine is part of the bodys digestive system, which also includes the esophagus, stomach, and large intestine. The digestive system rem

questions answers What are the treatments for Small Intestine Cancer ? Key Points
 - There are different types of treatment for patients with small intestine cancer. - Three types of standard treatment are used: - Surgery - Radiation therapy - Chemotherapy - New types of treatment are being tested in clinical trials. - Biologic therapy - Radiation therapy with radiosensitizers - Patients may want to think about taking part in a clinical trial. - Patients can enter clinical trials before, during, or after starting their cancer treatment. - Follow-up tests may be needed.
 
 
 There are different types of treatment for patients with small intestine cancer.
 Different types of treatments are available for patients with small intestine cancer. Some treatments are standard (the currently used treatment), and some are being tested in clinical trials. A treatment clinical trial is a research study meant to help improve current treatments or obtain information on new treatments for patients with cancer. When clinical trials show that a new treatment is better than the standard treatment, the new treatment may become the standard treatment. Patients may want to think about taking part in a clinical trial. Some clinical trials are open only to patients who have not started treatment.
 
 
 Three types of standard treatment are used:
 Surgery Surgery is the most common treatment of small intestine cancer. One of the following types of surgery may be done: - Resection: Surgery to remove part or all of an organ that contains cancer. The resection may include the small intestine and nearby organs (if the cancer has spread). The doctor may remove the section of the small intestine that contains cancer and perform an anastomosis (joining the cut ends of the intestine together). The doctor will usually remove lymph nodes near the small intestine and examine them under a microscope to see whether they contain cancer. - Bypass: Surgery to allow food in the small intestine to go around (bypass) a tumor that is blocking the intestine but cannot be removed. Even if the doctor removes all the cancer that can be seen at the time of the surgery, some patients may be given radiation therapy after surgery to kill any cancer cells that are left. Treatment given after the surgery, to lower the risk that the cancer will come back, is called adjuvant therapy. Radiation therapy Radiation therapy is a cancer treatment that uses high-energy x-rays or other types of radiation to kill cancer cells or keep them from growing. There are two types of radiation therapy: - External radiation therapy uses a machine outside the body to send radiation toward the cancer. - Internal radiation therapy uses a radioactive substance sealed in needles, seeds, wires, or catheters that are placed directly into or near the cancer. The way the radiation therapy is given depends on the type of the cancer being treated. External radiation therapy is used to treat small intestine cancer. Chemotherapy Chemotherapy is a cancer treatment that uses drugs to stop the growth of cancer cells, either by killing the cells or by stopping them from dividing. When chemotherapy is taken by mouth or injected into a vein or muscle, the drugs enter the bloodstream and can reach cancer cells throughout the body (systemic chemotherapy). When chemotherapy is placed directly into the cerebrospinal fluid, an organ, or a body cavity such as the abdomen, the drugs mainly affect cancer cells in those areas (regional chemotherapy). The way the chemotherapy is given depends on the type and stage of the cancer being treated.
 
 
 New types of treatment are being tested in clinical trials.
 This summary section describes treatments that are being studied in clinical trials. It may not mention every new treatment being studied. Information about clinical trials is available from the NCI website. Biologic therapy Biologic therapy is a treatment that uses the patient's immune system to fight cancer. Substances made b

## Done